# Ex 1.1 Motion Detection & Tracking
This task focuses on developing an application to detect and track moving cars from a camera recording. The algorithm of the application is based on the frame differencing and background subtraction techniques. The application will only focus on detecting cars that are in the "Main street".

In [1]:
# IMPORTING LIBRARIES FOR PROJECT
import cv2
import numpy as np

The traffic video ```Traffic_Laramie_1.mp4``` is loaded for verifying results.

In [2]:
# loading the video file
vid_path = "media/Traffic_Laramie_1.mp4"
capture = cv2.VideoCapture(vid_path)

# checking if video is opened successfully
if not capture.isOpened(): print("Error: Could not open video.")
else: print("Successfully opened video.")

Successfully opened video.


OpenCV will be used to implement the background subtraction in the algorithm.

This project will use the updated ```createBackgroundSubtractorMOG2()```
- ```history``` is the length of the history
- ```varThreshold``` is the threshold on the squared Mahalanobis distance between the pixel and the model to decide whether a pixel is well described by the background model
- ```detectShadows``` checks for shadows and marks them

The base values are used, and will be modified to better detect the cars.

In [3]:
background_sub = cv2.createBackgroundSubtractorMOG2( history = 500,
                                                     varThreshold = 50,
                                                     detectShadows = True )

Now, the algorithm will read the input video frame-by-frame, and apply background subtraction to extract the moving objects. Morphological operations will be done to clean the mask, and the algorithm will detect contours to find moving cars. This is then displayed by drawing a bounding box around detected cars.

Note that because this project focuses on motion tracking, the frames will be converted to grayscale to help simplify preprocessing when conducting Background Subtraction using OpenCV.

In [4]:
while True:
    check, img = capture.read()
    if not check:
        break # stop if end of video
        
    # converting to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # applying background subtraction
    fg_mask = background_sub.apply(gray)
    
    # applying thresholding to clean up noise
    threshold_mask = cv2.threshold(fg_mask, 150, 255, cv2.THRESH_BINARY)[1]
    # finding contours to detect moving objects
    contours = cv2.findContours(threshold_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    for c in contours:
        # contourArea() filters out small contours
        if cv2.contourArea(c) < 1000:
            continue
        
        # drawing bounding box around detected cars
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # displaying the results
    cv2.imshow("Traffic Video", img)
    # cv2.imshow("Foreground Mask", fg_mask)
    # cv2.imshow("Threshold Mask", threshold_mask)
    
    # exiting the loop
    key = cv2.waitKey(1)
    # using "q" key to exit
    if key == ord("q"):
        break

# after the loop, release the video object
capture.release()
# destroy all windows
cv2.destroyAllWindows()